In [1]:
# Install packages:
# !pip install pandas
# !pip install folium
# !pip install numpy

In [2]:
class Node:
    def __init__(self, street_name, lat, lon): # string, float, float
        self.street_name = street_name # string
        self.lat = lat # float
        self.lon = lon # float
        self.neighbors = [] # array of (Node, float)
        self.neighbor_names = set() # set of create_key
    
    def add_neighbor(self, end_node, weight):  # Node object, float
        if end_node.create_key() not in self.neighbor_names:
            neighbor = (end_node, weight)
            self.neighbors.append(neighbor)
            self.neighbor_names.add(end_node.create_key())
    
    def create_key(self):
        return str(self.lat) + "," + str(self.lon)
    
    @staticmethod
    def create_key_static(lat, lon):
        return str(lat) + "," + str(lon)
            
    def __str__(self):
        neighbor_str = ""
        
        for n in self.neighbors:
            neighbor_str += f"({n[0].street_name}, {n[0].lat}, {n[0].lon})"
        
        return "STREET NAME: " + self.street_name + ", WITH NEIGHBORS: " + neighbor_str

class Graph:
    def __init__(self):
        self.graph = {} # map from create_key(node): Node
        self.size = 0 # integer
    
    # if vertex exists, returns True, otherwise False.
    def add_vertex(self, street_name, lat, lon): # string, float, float
        new_node = Node(street_name, lat, lon)
        key = new_node.create_key()
        
        if key not in self.graph:
            self.graph[key] = new_node
            self.size += 1
            return True
        else:
            # vertex already exists.
            return False
    
    def add_edge_with_keys(self, start_key, end_key, weight=0): # for when you re-create the graph
        if start_key in self.graph and end_key in self.graph:
            self.graph[start_key].add_neighbor(self.graph[end_key], weight)
            self.graph[end_key].add_neighbor(self.graph[start_key], weight)
            return True
        else:
            return False
        
    def add_edge(self, start_lat, start_lon, end_lat, end_lon, weight=0): # create_key_static, create_key_static, float
        start_key = Node.create_key_static(start_lat, start_lon)
        end_key = Node.create_key_static(end_lat, end_lon)
           
        if start_key in self.graph and end_key in self.graph:
            self.graph[start_key].add_neighbor(self.graph[end_key], weight)
            self.graph[end_key].add_neighbor(self.graph[start_key], weight)
            return True
        else:
            return False

    def print(self):
        for key in self.graph:
            print(key, self.graph[key])
            
    def export_nodes(self):
        result = []
        
        for k in self.graph:
            key = k
            lat = self.graph[key].lat
            lon = self.graph[key].lon
            street_name = self.graph[key].street_name
            node = {
                "key": key,
                "lat": lat,
                "lon": lon,
                "street_name": street_name
            }
            result.append(node)
        
        return result
    
    def export_edges(self):
        result = []
        
        for k in self.graph:
            neighbors = self.graph[k].neighbors
            
            for n in neighbors:
                key1 = k
                key2 = (n[0].create_key())         
                weight = (n[1])
                node = {
                    "key1": key1,
                    "key2": key2,
                    "weight": weight
                }
                result.append(node)
        
        return result
        
# test graph
graph = Graph()

In [66]:
import pandas as pd
import folium
import numpy as np

# read in data and get coordinates:
df = pd.read_csv('real_data.csv')
df["latitude_longitude"] = df["latitude_longitude"].str.strip()
df["latitude_longitude"]= df["latitude_longitude"].str.split(" ", n = -1, expand = False)
df["latitude_longitude"] = df["latitude_longitude"][::-1]
coordinates = df["latitude_longitude"].tolist()

graph = Graph()

for coords in coordinates:
    points = coords[::-1]

    for i in range(0, len(points) - 2, 2):
        lat = points[i]
        lon = points[i + 1]
        street_name_1 = df["street"][test_index]
        next_point_lat = points[i + 2]
        next_point_lon = points[i + 3]
        street_name_2 = df["street"][test_index+1]
        
        graph.add_vertex(street_name_1, lat, lon)
        graph.add_vertex(street_name_2, next_point_lat, next_point_lon)
        next_point_lat = points[i + 2]
        next_point_lon = points[i + 3]
        
        distance = np.sqrt((float(next_point_lat) - float(lat))**2 + (float(next_point_lon) - float(lon))**2)
        graph.add_edge(lat, lon, next_point_lat, next_point_lon, distance)

# only need to run once - create the graph and export to JSON files for nodes and edges:

# edges = graph.export_edges()
# nodes = graph.export_nodes()

# import json

# with open('edges.json', 'w') as fout:
#     json.dump(edges, fout)
    
# with open('nodes.json', 'w') as fout:
#     json.dump(nodes, fout)

In [ ]:
# read in the files and re-create the graph.
import json

edges_file = open('edges.json')
vertices_file = open('nodes.json')

edges = json.load(edges_file)
vertices = json.load(vertices_file)

graph = Graph()

for o in vertices:
    graph.add_vertex(o['street_name'], float(o['lat']), float(o['lon']))

for o in edges:
    graph.add_edge_with_keys(o['key1'], o['key2'], float(o['weight']))

In [68]:
from queue import PriorityQueue

def shortest_path(g, start):
    # other node: shortest distance from source node (map)
    distances = {}
    
    # map from current node to previous node
    previous = {}
    
    # priority queue of (vertex, float)
    pq = PriorityQueue()
    
    # set of strings
    visited = set()
    
    for vertex in g.graph.keys():
        distances[vertex] = float('inf')
        previous[vertex] = None
        
        if vertex != start:
            pq.put((vertex,))
    
    distances[start] = 0
    
    while not pq.empty():
        current_node = pq.get()
        
        for n in g.graph[current_node].neighbors:
            neighbor = n[0]
            distance = n[1]
            
            tempDistance = distances[neighbor.create_key()] + distance
            
            if tempDistance < distances[vertex]:
                distances[vertex] = tempDistance
                previous[vertex] = neighbor
    
    for k in distances.keys():
        if distances[k] != float('inf'):
            print(distances[k])
    
    return distances, previous